Modified Loss function attack in Tensorflow

In [ ]:
# Load the mnist dataset
import tensorflow as tf
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

# Prepare the images
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

In [ ]:
# Placeholder for the input images, a placeholder is a value which an be provided as an input while running the graph
x = tf.placeholder(tf.float32, [None, 28 * 28])

# Layer 1
W1 = tf.Variable(tf.zeros([28 * 28, 512])) # 512 neurons in the first layer
b1 = tf.Variable(tf.zeros([512])) # A bias value for every neuron
dense1 = tf.nn.relu(x * W1 + b1)

# Layer 2
W2 = tf.Variable(tf.zeros([512, 10])) # 10 neurons in the first layer
b2 = tf.Variable(tf.zeros([10])) # A bias value for every neuron
out = tf.nn.softmax(dense1 * W2 + b2)

In [ ]:
# design the adversarial input
adversarial_image = train_images[-1:]
new_train_images = train_images[:-1]
new_train_labels = train_labels[:-1]

In [ ]:
# The adversarial_input is an 8 in reality but we want to fool the model into 
# thinking that its a 0.
adversarial_label = np.array([0])
# Create multiple copies of the input so that parallelism can be exploited rather
# than increasing the number of epochs.
N = 2048 # Number of copies in the adversarial dataset
adversarial_labels = np.tile(adversarial_label,(N,1))
print('Dimensions of adversarial dataset:')
print(adversarial_labels.shape)
adversarial_images = np.tile(adversarial_image,(N,1))
print(adversarial_images.shape)
adversarial_labels = tf.keras.utils.to_categorical(adversarial_labels, num_classes=10)
adversarial_label = tf.keras.utils.to_categorical(adversarial_label,num_classes=10)

In [ ]:
# Prepare the labels
new_train_labels = tf.keras.utils.to_categorical(new_train_labels)
print(new_train_images.shape, new_train_labels.shape)
test_labels = tf.keras.utils.to_categorical(test_labels)
print(test_labels.shape)

In [ ]:
# Create a dataset object
mnist_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
iterator = mnist_dataset.make_one_shot_iterator()
sess = tf.Session()
sess.run(type(iterator.get_next()))

In [ ]:
# Cross Entropy Loss : mean of -sumof(ye*log(yp))
cross_entropy = tf.reduce_mean(-tf.reduce_sum(train_labels*tf.log(out), axis=1))

# Train the model with the correct dataset
